In [1]:
import os
import time

from CAJAL.lib import run_gw, sample_swc

# Run GW on sampled points using scipy dist metric (usually Euclidean)

In [2]:
# Sample points from neuron reconstruction files
start = time.time()
infolder = "../CAJAL/data/swc_files"
outfolder = "../CAJAL/data/sampled_pts/swc_sampled_50"
sample_swc.save_sample_pts_parallel(infolder, outfolder, types_keep=[0, 1, 2, 3, 4],
                                    goal_num_pts=50, num_cores=8)
print('Total time elapsed: {:.3f}s'.format(time.time() - start))

/home/patn/CAJAL/CAJAL/lib/sample_swc.py:161: UserWarning: More disconnected segments in neuron than points to sample, skipping
  warnings.warn("More disconnected segments in neuron than points to sample, skipping")
/home/patn/CAJAL/CAJAL/lib/sample_swc.py:161: UserWarning: More disconnected segments in neuron than points to sample, skipping
  warnings.warn("More disconnected segments in neuron than points to sample, skipping")
/home/patn/CAJAL/CAJAL/lib/sample_swc.py:161: UserWarning: More disconnected segments in neuron than points to sample, skipping
  warnings.warn("More disconnected segments in neuron than points to sample, skipping")
/home/patn/CAJAL/CAJAL/lib/sample_swc.py:161: UserWarning: More disconnected segments in neuron than points to sample, skipping
  warnings.warn("More disconnected segments in neuron than points to sample, skipping")
/home/patn/CAJAL/CAJAL/lib/sample_swc.py:161: UserWarning: More disconnected segments in neuron than points to sample, skipping
  warnin

Total time elapsed: 46.466s


In [3]:
# Set parameters
data_dir = os.path.abspath('../CAJAL/data/sampled_pts/swc_sampled_50/')
gw_results_dir = os.path.abspath('../CAJAL/data/gw_results')
data_prefix = "a10_full" # Not required, but helpful if another file (like python_list_dir.txt) is in same folder
num_cores = 12
file_prefix = "a10_full_euclidean"

In [5]:
# Compute Euclidean distance on sampled points, then compute and save GW distance between cells
t1 = time.time()
print("Calculating distances between points in each dataset")
dist_mat_list = run_gw.get_intracell_distances_all(data_dir=data_dir, data_prefix=data_prefix)
t2 = time.time()
print("Calculating GW distances between datasets")
run_gw.compute_and_save_GW_dist_mat(dist_mat_list, file_prefix, gw_results_dir, save_mat=True, num_cores=num_cores)
t3 = time.time()
print('Time to compute distances: {:.3f}s'.format(t2 - t1))
print('Time in GW calculation: {:.3f}s'.format(t3 - t2))
print('Total time elapsed: {:.3f}s'.format(t3 - t1))

Calculating distances between points in each dataset
Calculating GW distances between datasets
Time to compute distances: 0.192s
Time in GW calculation: 7.003s
Total time elapsed: 7.195s


# Run GW on precomputed distance (usually geodesic)

In [6]:
# Save geodesic network distance on sampled points from neuron reconstruction files
start = time.time()
infolder = "../CAJAL/data/swc_files"
outfolder = "../CAJAL/data/sampled_pts/swc_geodesic_50"
sample_swc.save_geodesic_parallel(infolder, outfolder, types_keep=[0, 1, 2, 3, 4],
                                  goal_num_pts=50, num_cores=8)
print('Total time elapsed: {:.3f}s'.format(time.time() - start))

Total time elapsed: 50.909s


In [7]:
# Set parameters
distances_dir = os.path.abspath('../CAJAL/data/sampled_pts/swc_geodesic_50/')
gw_results_dir = os.path.abspath('../CAJAL/data/gw_results')
data_prefix = "a10_full" # Not required, but helpful if another file (like python_list_dir.txt) is in same folder
num_cores = 12
file_prefix = "a10_full_geodesic"

In [9]:
# Load geodesic distances, then compute and save GW distance between cells
t1 = time.time()
print("Calculating distances between points in each dataset")
dist_mat_list = run_gw.load_distances_global(distances_dir=distances_dir, data_prefix=data_prefix)
t2 = time.time()
print("Calculating GW distances between datasets")
run_gw.compute_and_save_GW_dist_mat(dist_mat_list, file_prefix, gw_results_dir, save_mat=True, num_cores=num_cores)
t3 = time.time()
print('Time to load distances: {:.3f}s'.format(t2 - t1))
print('Time in GW calculation: {:.3f}s'.format(t3 - t2))
print('Total time elapsed: {:.3f}s'.format(t3 - t1))

Calculating distances between points in each dataset
Calculating GW distances between datasets
Time to load distances: 0.082s
Time in GW calculation: 5.209s
Total time elapsed: 5.290s


# Save python_list_dir
R and Python sort strings differently, so I save the python ordering in case I need to load the distance matrix into R

In [10]:
pj = lambda *paths: os.path.abspath(os.path.join(*paths))
distances_dir = os.path.abspath("../CAJAL/data/sampled_pts/swc_sampled_50")
data_prefix = "a10_full"
files_list = os.listdir(distances_dir)
files_list = [data_file for data_file in files_list\
              if data_prefix is None or data_file.startswith(data_prefix)]
files_list.sort() # sort the list because sometimes os.listdir() result is not sorted
list_dir_file = open(pj(distances_dir,"python_list_dir.txt"), "w+")
for file_name in files_list:
    list_dir_file.write(file_name+"\n")
list_dir_file.close()